# Fetch results 

In [ ]:
options(warn=-1)

In [ ]:
library_load <- suppressMessages(
    
    suppressWarnings(
        
        list(
        
            # Seurat 
            library(Seurat), 

            # Data 
            library(tidyverse), 
            library(data.table), 

            # miloR
            library(miloR), 
            library(ggbeeswarm), 

            # Plotting 
            library(ggplot2), 
            library(patchwork), 
            library(cowplot), 

            # Pyhton compatibility
            library(reticulate)

        )
    )
)

In [ ]:
# Configure reticulate 
use_condaenv(condaenv="pyscenic.0.12.1", conda="/nobackup/peer/fdeckert/miniconda3/bin/conda", required=NULL)
py_config()

In [ ]:
random_seed <- 42
set.seed(random_seed)

In [ ]:
# Set working directory to project root
setwd("/research/peer/fdeckert/FD20200109SPLENO")

In [ ]:
# Plotting Theme
source("plotting_global.R")
ggplot2::theme_set(theme_global_set(size_select=1)) # From project global source()

In [ ]:
# Helper functions
source("bin/so_pl.R")
source("bin/dp_feature.R")

# Import data 

In [ ]:
meta <- read.csv("data/bulkRNAseq/object/meta.csv", row.names=1)
meta$sample_group <- paste0(meta$genotype, "_", meta$disease, "_", meta$dpi)

In [ ]:
auc_mtx <- read.csv("result/scenic/bulkRNAseq/auc_mtx.csv", row.names=1)
colnames(auc_mtx) <- gsub("\\.", "", colnames(auc_mtx))

In [ ]:
meta <- meta[intersect(rownames(meta), rownames(auc_mtx)), ]
auc_mtx <- auc_mtx[intersect(rownames(meta), rownames(auc_mtx)), ]

In [ ]:
# Load python libraries
pysenic_rss <- import("pyscenic.rss", convert=FALSE)
pd <- import("pandas", convert=FALSE)

In [ ]:
rss_get <- function(meta, auc_mtx, sample_group) {
    
    # Load python libraries
    pysenic_rss <- import("pyscenic.rss", convert=FALSE)
    pd <- import("pandas", convert=FALSE)

    # Get meta subset
    meta <- meta[meta$sample_group==sample_group, ]

    # Get AUC matx 
    auc_mtx <- auc_mtx[rownames(meta), ]

    # Get regulon specific scores 
    rss <- pysenic_rss$regulon_specificity_scores(auc_mtx, pd$Series(r_to_py(meta$celltype))) %>% py_to_r()

    # Get matrix
    mat <- tibble::rownames_to_column(rss, "celltype") %>% 
        reshape2::melt(., id.vars=c("celltype"), variable.name="module", value.name="rss") %>% 
        dplyr::arrange(celltype, desc(rss)) %>% 
        dplyr::group_by(celltype) %>%
        dplyr::mutate(rank=1:n()) %>% 
        dplyr::mutate(label_rank=ifelse(rank %in% 1:15, as.character(module), NA)) %>% 
        dplyr::mutate(sample_group=sample_group)

    return(mat)
    
}

In [ ]:
rss_plot <- function(mat, title=NULL) {

    dp_rss_top_label <- ggplot(mat, aes(x=rank, y=rss, label=label_rank, color=celltype)) + 
        geom_point(size=1) + 
        xlab("") + ylab("RSS") + ggtitle(title) + 
        ylim(c(min(mat$rss), max(mat$rss) + 0.1)) + 
        facet_grid(~celltype, scales="free_y") +
        scale_color_manual(values=color$celltype, name="Cell type") + 
        scale_x_continuous(breaks=seq(0, floor(max(mat_1$rank)/100)*100, 100), labels=seq(0, floor(max(mat$rank)/100)*100, 100)) + 
        ggrepel::geom_text_repel(segment.color="black", force=10, force_pull=1, max.overlaps=getOption("ggrepel.max.overlaps", default=100), size=5, alpha=1, guide="none", segment.size=0.05, color="black", min.segment.length=0) + 
        guides(color=guide_legend(ncol=1, override.aes=list(alpha=1, size=1.5), keywidth=0, keyheight=0.25, default.unit="cm")) + 
        facet_wrap(~celltype, nrow=1)

    return(dp_rss_top_label)
    
}

In [ ]:
mat_1 <- rss_get(meta, auc_mtx, "IFNAR_fl_Baseline_D0")
mat_2 <- rss_get(meta, auc_mtx, "IFNAR_fl_CpG_D1")
mat_3 <- rss_get(meta, auc_mtx, "IFNAR_fl_CpG_D3")
mat_4 <- rss_get(meta, auc_mtx, "IFNAR_fl_CpG_D6")
mat_5 <- rss_get(meta, auc_mtx, "IFNAR_fl_LysM_cre_Baseline_D0")
mat_6 <- rss_get(meta, auc_mtx, "IFNAR_fl_LysM_cre_CpG_D1")
mat_7 <- rss_get(meta, auc_mtx, "IFNAR_fl_LysM_cre_CpG_D3")
mat_8 <- rss_get(meta, auc_mtx, "IFNAR_fl_LysM_cre_CpG_D6")

In [ ]:
saveRDS(list(IFNAR_fl_Baseline_D0=mat_1, IFNAR_fl_CpG_D1=mat_2, IFNAR_fl_CpG_D3=mat_3, IFNAR_fl_CpG_D6=mat_4, IFNAR_fl_LysM_cre_Baseline_D0=mat_5, IFNAR_fl_LysM_cre_CpG_D1=mat_6, IFNAR_fl_LysM_cre_CpG_D3=mat_7, IFNAR_fl_LysM_cre_CpG_D6=mat_8), "data/bulkRNAseq/scenic/mat.rds")

In [ ]:
rss_plot(mat_1, "Ifnar fl/fl (Baseline)")
rss_plot(mat_2, "Ifnar fl/fl (CpG D1)")
rss_plot(mat_3, "Ifnar fl/fl (CpG D3)")
rss_plot(mat_4, "Ifnar fl/fl (CpG D6)")
rss_plot(mat_5, "Ifnar fl/fl LysM Cre (Baseline)")
rss_plot(mat_6, "Ifnar fl/fl LysM Cre (CpG D1)")
rss_plot(mat_7, "Ifnar fl/fl LysM Cre (CpG D3)")
rss_plot(mat_8, "Ifnar fl/fl LysM Cre (CpG D6)")